# Clustering Crypto

In [7]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import json

### Fetching Cryptocurrency Data

In [11]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
json_request = Request(url)
json_response = urlopen(json_request)
data = json_response.read()
json_data = json.loads(data)

In [12]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
crypto_df = pd.DataFrame(json_data['Data']).T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,BlockTime,AssetLaunchDate,MaxSupply,MktCapPenalty,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,IsUsedInDefi
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,0,2017-07-01,300,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame

### Data Preprocessing

In [13]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
slim_crypto_df = crypto_df.drop(columns=['Rating', 'Url', 'Taxonomy', 'SortOrder', 'Sponsored', 'SmartContractAddress', 'Id', 'BuiltOn', 'FullName', 'ImageUrl', 'ContentCreatedOn'])

In [14]:
# Keep only cryptocurrencies that are trading
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['IsTrading'] == True]

In [15]:
# Keep only cryptocurrencies with a working algorithm
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['Algorithm'] != 'N/A']

In [16]:
# Remove the "IsTrading" column
slim_crypto_df = slim_crypto_df.drop(columns='IsTrading')

In [17]:
# Remove rows with at least 1 null value
slim_crypto_df = slim_crypto_df.dropna()

In [18]:
# Remove rows with cryptocurrencies having no coins mined
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['TotalCoinsMined'] != 0]

In [20]:
# Drop rows where there are 'N/A' text values
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['ProofType'] != 'N/A']

In [21]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
names = slim_crypto_df

In [22]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
slim_crypto_df = slim_crypto_df.drop(columns='CoinName')

In [25]:
# Create dummy variables for text features
slim_crypto_df['BlockNumber'] = slim_crypto_df.BlockNumber.astype(float)
slim_crypto_df['BlockReward'] = slim_crypto_df.BlockReward.astype(float)
slim_crypto_df['BlockTime'] = slim_crypto_df.BlockTime.astype(float)
slim_crypto_df['NetHashesPerSecond'] = slim_crypto_df.NetHashesPerSecond.astype(float)
slim_crypto_df['TotalCoinsMined'] = slim_crypto_df.TotalCoinsMined.astype(float)

In [26]:
slim_crypto_df.dtypes

Name                   object
Symbol                 object
Description            object
AssetTokenStatus       object
Algorithm              object
ProofType              object
TotalCoinsMined       float64
BlockNumber           float64
NetHashesPerSecond    float64
BlockReward           float64
BlockTime             float64
AssetLaunchDate        object
MaxSupply              object
MktCapPenalty          object
PlatformType           object
DecimalPoints          object
Difficulty             object
IsUsedInDefi           object
dtype: object

In [29]:
# Standardize data
slim_crypto_encoded = pd.get_dummies(data=slim_crypto_df, columns=['Name', 'Algorithm', 'ProofType', 'Symbol'])
slim_crypto_df_scaled = StandardScaler().fit_transform(slim_crypto_encoded)
slim_crypto_df_scaled

ValueError: could not convert string to float: 'Bitcoin uses peer-to-peer technology to operate with no central authority or banks; managing transactions and the issuing of bitcoins is carried out collectively by the network. Although other cryptocurrencies have come before, Bitcoin is the first decentralized cryptocurrency - Its reputation has spawned copies and evolution in the space.With the largest variety of markets and the biggest value - having reached a peak of 318 billion USD - Bitcoin is here to stay. As with any new invention, there can be improvements or flaws in the initial model however the community and a team of dedicated developers are pushing to overcome any obstacle they come across. It is also the most traded cryptocurrency and one of the main entry points for all the other cryptocurrencies. The price is as unstable as always and it can go up or down by 10%-20% in a single day.Bitcoin is an SHA-256 POW coin with almost 21,000,000 total minable coins. The block time 

### Reducing Dimensions Using PCA

In [28]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(slim_crypto_df_scaled)

NameError: name 'slim_crypto_df_scaled' is not defined

In [ ]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=slim_crypto_df.index
)
pca_df.head()

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Visualization", xlabel="Number of Clusters", ylabel="Inertia")

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model
kmodel = KMeans(n_clusters=6, random_state=0)

# Fit the model
kmodel.fit(pca_df)

# Predict clusters
predictions = kmodel.predict(pca_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.DataFrame({
    "CoinName": names.CoinName,
    "Algorithm": slim_crypto_df.Algorithm,
    "ProofType": slim_crypto_df.ProofType,
    "TotalCoinsMined": slim_crypto_df.TotalCoinsMined,
    "TotalCoinSupply": slim_crypto_df.TotalCoinSupply,
    "PC 1": pca_df['PC 1'],
    "PC 2": pca_df['PC 2'],
    "PC 3": pca_df['PC 3'],
    "Class": kmodel.labels_,
    "Predictions": predictions, 
    },
)
clustered_df.head()

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name='CoinName',
    hover_data= ['Algorithm'],
    color="Class",
    symbol="Class",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
clustered_df.hvplot.table(columns)

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot
clustered_df['TotalCoinsMined'] = clustered_df['TotalCoinsMined'].astype(float) / 1000000000
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(float) / 1000000000

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='Class', 
    colormap="Inferno_r", 
    hover_cols=['CoinName']
)